<a href="https://colab.research.google.com/github/JoaoPedro-repertoriodeexatas/PS_Ligia_Time-06/blob/main/Tratamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

Este estudo retrospectivo transversal analisou 1267 registros de pacientes adultos admitidos em dois departamentos de emergência entre outubro de 2016 e setembro de 2017. Foram avaliadas vinte e quatro variáveis, incluindo queixas principais, sinais vitais registrados pela enfermagem e desfechos clínicos. Dois especialistas em triagem (uma enfermeira de emergência certificada na chegada do paciente e um provedor e instrutor final de KTAS para avaliação correta) determinaram o KTAS 'verdadeiro' de cada caso.


# Objetivo

O objetivo deste estudo é analisar as características dos pacientes, o processo de triagem e os desfechos clínicos em departamentos de emergência, utilizando dados que incluem queixas principais, sinais vitais e categorizações de triagem por especialistas. Visa-se também a compreensão do significado das variáveis originais do dataset para extensão fundamental para modelagem dos dados.

# Variáveis atribuídas ao dataset original


O dataset contém as seguintes variáveis:

*   **Sex**: Sexo do paciente
    *   `1 = Feminino`
    *   `2 = Masculino`
*   **Age**: Idade do paciente
*   **Arrival mode**: Modo de transporte para o hospital
    *   `1 = Caminhando`
    *   `2 = Ambulância Pública`
    *   `3 = Veículo Privado`
    *   `4 = Ambulância Privada`
    *   `5,6,7 = Outros`
*   **Injury**: Se o paciente está ferido
    *   `1 = Não`
    *   `2 = Sim`
*   **Chief_complain**: Queixa principal do paciente
*   **Mental**: Estado mental do paciente
    *   `1 = Alerta`
    *   `2 = Responde à Voz`
    *   `3 = Responde à Dor`
    *   `4 = Sem Resposta`
*   **Pain**: Se o paciente sente dor
    *   `1 = Sim`
    *   `0 = Não`
*   **NRS_pain**: Avaliação da dor pela enfermeira (Escala Numérica de Dor)
*   **SBP**: Pressão Arterial Sistólica
*   **DBP**: Pressão Arterial Diastólica
*   **HR**: Frequência Cardíaca
*   **RR**: Frequência Respiratória
*   **BT**: Temperatura Corporal
*   **Group**: Tipo de Departamento de Emergência (ED)
    *   `1 = ED Local (3º Grau)`
    *   `2 = ED Regional (4º Grau)`
*   **Disposition**: Desfecho do paciente
    *   `1 = Alta`
    *   `2 = Internação (Enfermaria)`
    *   `3 = Internação (UTI)`
    *   `4 = Alta`
    *   `5 = Transferência`
    *   `6 = Óbito`
    *   `7 = Cirurgia`
*   **KTAS_RN**: Pontuação KTAS determinada pela Enfermeira
*   **KTAS_expert**: Pontuação KTAS determinada pelo Especialista
    *   `1,2,3 = Emergência`
    *   `4,5 = Não Emergência`
*   **mistriage**: Indicação de erro de triagem
    *   `0 = Sem Erro`
    *   `1 = Sub-triagem (paciente mais grave do que o classificado)`
    *   `2 = Super-triagem (paciente menos grave do que o classificado)`

# Configuração e Carregamento dos Dados

 A primeira etapa essencial em qualquer pipeline de dados é garantir que as bibliotecas necessárias estejam disponíveis e que os dados sejam ingeridos corretamente. Aqui, utilizamos o pandas para manipulação tabular e numpy para operações numéricas. Um ponto crítico de atenção é o encoding do arquivo CSV original; o uso de latin1 é necessário para interpretar corretamente caracteres especiais da língua, evitando erros de leitura. Além disso, tratamos imediatamente valores de ruído conhecidos, como '#BOŞ!', que representam dados faltantes na fonte original.

In [ ]:
import pandas as pd
import numpy as np

# URL raw do arquivo no GitHub
url_raw = 'https://raw.githubusercontent.com/Cosmo445/triage-decision-support/refs/heads/main/data.csv'

# Carregamento e limpeza inicial
df = pd.read_csv(
    url_raw,
    sep=';',
    encoding='latin1'
)

# Tratamento de ruído: substituição de string de erro por NaN
df.replace('#BOŞ!', np.nan, inplace=True)

print(f"Dataset carregado com {df.shape[0]} linhas e {df.shape[1]} colunas.")
df.head()

Dataset carregado com 1267 linhas e 24 colunas.


,Group,Sex,Age,Patients number per hour,Arrival mode,Injury,Chief_complain,Mental,Pain,NRS_pain,...,BT,Saturation,KTAS_RN,Diagnosis in ED,Disposition,KTAS_expert,Error_group,Length of stay_min,KTAS duration_min,mistriage
0,2,2,71,3,3,2,right ocular pain,1,1,2,...,36.6,100,2,Corneal abrasion,1,4,2,86,"5,00",1
1,1,1,56,12,3,2,right forearm burn,1,1,2,...,36.5,NaN,4,"Burn of hand, firts degree dorsum",1,5,4,64,"3,95",1
2,2,1,68,8,2,2,"arm pain, Lt",1,1,2,...,36.6,98,4,"Fracture of surgical neck of humerus, closed",2,5,4,862,"1,00",1
3,1,2,71,8,1,1,ascites tapping,1,1,3,...,36.5,NaN,4,Alcoholic liver cirrhosis with ascites,1,5,6,108,"9,83",1
4,1,2,58,4,3,1,"distension, abd",1,1,3,...,36.5,NaN,4,Ascites,1,5,8,109,"6,60",1


# 2. Padronização e Correção de Tipos Numéricos

Dados do mundo real frequentemente apresentam inconsistências de formatação. Neste dataset, variáveis vitais como Pressão Arterial e Saturação foram armazenadas como texto devido ao uso de vírgulas como separadores decimais (padrão local), enquanto o Python/Pandas espera pontos (padrão internacional). Esta etapa converte implicitamente essas colunas para numérico (float), permitindo cálculos estatísticos subsequentes. O uso de errors='coerce' é uma prática defensiva: qualquer valor que não possa ser convertido vira NaN, garantindo que o código não quebre por causa de um único erro de digitação.

In [ ]:
# Correção de tipos numéricos (substituindo vírgula por ponto)
colunas_numericas = ['NRS_pain', 'SBP', 'DBP', 'HR', 'RR', 'BT', 'Saturation', 'KTAS duration_min']
for col in colunas_numericas:
    # Verifica se é object (string) antes de tentar string methods
    if df[col].dtype == 'object':
        df[col] = df[col].str.replace(',', '.')

    # Converte para numérico, transformando erros em NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')
print("Tipos de colunas corrigidos.")
df[colunas_numericas].dtypes

Tipos de colunas corrigidos.


,0
NRS_pain,float64
SBP,float64
DBP,float64
HR,float64
RR,float64
BT,float64
Saturation,float64
KTAS duration_min,float64


# 3. Tratamento de Dados Faltantes (Imputação)

Modelos de Machine Learning e análises estatísticas não toleram valores vazios.

*   Variáveis Vitais (Numéricas): Optamos pela mediana em vez da média. A mediana é uma medida de tendência central robusta a outliers (valores extremos), que são comuns em dados de emergência (ex: uma pressão arterial 0 ou 300 pode distorcer a média, mas afeta pouco a mediana).
*   Diagnóstico (Categórico): Mantemos a integridade dos dados preenchendo vazios com a tag explícita 'Não informado', preservando a informação de que o dado estava ausente na origem.

In [ ]:
# Tratamento de valores nulos (Imputação pela Mediana)
# Realizado ANTES das categorizações para garantir que todos os pacientes tenham classificação
for col in colunas_numericas:
    mediana = df[col].median()
    df[col] = df[col].fillna(mediana)
# Preencher diagnósticos vazios com termo genérico para análise categórica
df['Diagnosis in ED'] = df['Diagnosis in ED'].fillna('Não informado')
print("Valores nulos tratados.")
df.isnull().sum()[colunas_numericas]

Valores nulos tratados.


,0
NRS_pain,0
SBP,0
DBP,0
HR,0
RR,0
BT,0
Saturation,0
KTAS duration_min,0


# 4. Tradução e Enriquecimento Semântico

Códigos numéricos (1, 2, 3...) são eficientes para armazenamento, mas péssimos para análise humana e visualização. Nesta etapa, aplicamos dicionários de mapeamento para traduzir códigos em rótulos legíveis (ex: Sexo 1 -> 'Feminino'). Isso facilita a interpretação imediata dos gráficos de EDA (Exploratory Data Analysis) que serão gerados posteriormente.

In [ ]:
# Mapeamento de variáveis categóricas originais
df['Sexo_legenda'] = df['Sex'].map({1: 'Feminino', 2: 'Masculino'})
df['Ferimento_legenda'] = df['Injury'].map({1: 'Não', 2: 'Sim'})
df['Dor_legenda'] = df['Pain'].map({1: 'Sim', 0: 'Não'})
df['Estado_Mental_legenda'] = df['Mental'].map({
    1: 'Alerta', 2: 'Responde à Voz', 3: 'Responde à Dor', 4: 'Sem Resposta'
})
df['Tipo_Hospital_legenda'] = df['Group'].map({
    1: 'Local (3º Grau)', 2: 'Regional (4º Grau)'
})
mapa_chegada = {1: 'Caminhando', 2: 'Ambulância Pública', 3: 'Veículo Privado', 4: 'Ambulância Privada'}
df['Modo_Chegada_legenda'] = df['Arrival mode'].map(lambda x: mapa_chegada.get(x, 'Outros'))
mapa_desfecho = {
    1: 'Alta', 2: 'Internação (Enfermaria)', 3: 'Internação (UTI)',
    4: 'Alta', 5: 'Transferência', 6: 'Óbito', 7: 'Cirurgia'
}
df['Desfecho_legenda'] = df['Disposition'].map(mapa_desfecho)
print("Mapeamentos concluídos.")
df[['Sex', 'Sexo_legenda']].head()

Mapeamentos concluídos.


,Sex,Sexo_legenda
0,2,Masculino
1,1,Feminino
2,1,Feminino
3,2,Masculino
4,2,Masculino


# 5. Engenharia de Atributos: Classificações Clínicas

Aqui transformamos dados brutos em informação clínica acionável, baseando-nos em protocolos médicos estabelecidos.


*   Pressão Arterial: Utilizamos as faixas das Diretrizes Brasileiras de Hipertensão (SBC) para categorizar pacientes, identificando riscos cardiovasculares imediatos.

*   Sinais Vitais (FC, FR, Temp, SpO2): Criamos categorias de alerta (Taquicardia, Hipoxemia, Febre) que são muito mais informativas para triagem do que o valor numérico isolado.



In [ ]:
# Pressão Arterial (Baseado na Diretriz SBC)
def classificar_pressao(linha):
    pas, pad = linha['SBP'], linha['DBP']
    if pas >= 180 or pad >= 110: return 'Hipertensão Estágio 3'
    if 160 <= pas < 180 or 100 <= pad < 110: return 'Hipertensão Estágio 2'
    if 140 <= pas < 160 or 90 <= pad < 100: return 'Hipertensão Estágio 1'
    if 120 <= pas < 140 or 80 <= pad < 90: return 'Pré-hipertensão'
    return 'Normal'
df['Categoria_Pressao'] = df.apply(classificar_pressao, axis=1)
# Frequência Cardíaca
df['Categoria_FC'] = df['HR'].apply(
    lambda x: 'Taquicardia' if x > 100 else ('Bradicardia' if x < 60 else 'Normal')
)
# Frequência Respiratória
df['Categoria_FR'] = df['RR'].apply(
    lambda x: 'Taqupneia' if x > 20 else ('Bradipneia' if x < 12 else 'Normal')
)
# Temperatura Corporal
def classificar_temp(temp):
    if temp >= 37.8: return 'Febre'
    if temp >= 37.3: return 'Estado Febril'
    if temp >= 35.0: return 'Normal'
    return 'Hipotermia'
df['Categoria_Temperatura'] = df['BT'].apply(classificar_temp)
# Saturação de Oxigênio
def classificar_oxigenio(sat):
    if sat >= 95: return 'Normal'
    if sat >= 91: return 'Hipoxemia Leve'
    if sat >= 86: return 'Hipoxemia Moderada'
    return 'Hipoxemia Grave'
df['Categoria_Saturacao'] = df['Saturation'].apply(classificar_oxigenio)

# 6. Engenharia de Atributos: Métricas de Processo e Qualidade

Além da clínica, analisamos a eficiência operacional do pronto-socorro.

*   Qualidade da Triagem: Comparamos a triagem realizada (Enfermeiro) com a referência (Especialista) para detectar Mistriage (erros de classificação).

*   Fluxo e Tempos: Categorizamos o tempo de permanência e o fluxo de pacientes hora-a-hora, permitindo identificar gargalos operacionais ("Superlotação", "Demora na Triagem").

*   KTAS Simplificado: Binarizamos a escala KTAS em "Emergência" vs "Não Emergente" para facilitar modelos preditivos binários futuros.


In [ ]:
# Idade (Estatuto do Idoso)
df['Idade_Categoria'] = df['Age'].apply(lambda x: 'Adolescente' if x < 18 else ('Adulto' if x < 60 else 'Idoso'))
# Nível de Dor (Escala NRS)
def categorizar_dor_nivel(nrs):
    if nrs == 0: return 'Sem Dor'
    if nrs <= 3: return 'Dor Leve'
    if nrs <= 6: return 'Dor Moderada'
    return 'Dor Intensa'
df['Dor_Nivel'] = df['NRS_pain'].apply(categorizar_dor_nivel)
# Erros de Triagem
df['Erro_Tipo_Triagem'] = df['mistriage'].map({0: 'Correto', 1: 'Sub-triagem', 2: 'Super-triagem'})
df['Erro_Desvio_Magnitude'] = df['Error_group'].apply(
    lambda x: 'Sem Desvio' if x == 0 else ('Desvio Leve' if x <= 2 else 'Desvio Grave')
)
# Tempo de Permanência
def categorizar_permanencia(minutos):
    if minutos < 360: return 'Curta (até 6h)'
    if minutos < 1440: return 'Média (6h-24h)'
    return 'Internação (>24h)'
df['Permanencia_Categoria'] = df['Length of stay_min'].apply(categorizar_permanencia)
# Performance da Triagem
df['Triagem_Velocidade'] = df['KTAS duration_min'].apply(
    lambda x: 'Rápida' if x < 3 else ('Padrão' if x <= 7 else 'Demorada')
)
# Fluxo de Pacientes
def categorizar_fluxo(pacientes):
    if pacientes < 5: return 'Fluxo Baixo'
    if pacientes <= 10: return 'Fluxo Médio'
    return 'Superlotado'
df['Fluxo_Categoria'] = df['Patients number per hour'].apply(categorizar_fluxo)
# Binário KTAS: Emergência vs Não Urgente
def classificar_emergencia(val):
    return 'Emergência' if val in [1, 2, 3] else 'Não Urgente'
df['KTAS_Enfermeiro_classe'] = df['KTAS_RN'].apply(classificar_emergencia)
df['KTAS_Especialista_classe'] = df['KTAS_expert'].apply(classificar_emergencia)

# 7. Dataset Final Após o Pré-processamento

Aqui, apresentamos as colunas antigas tratadas junto com as novas colunas de categorias do DataFrame `df` após todas as etapas de limpeza, padronização, tratamento de nulos e engenharia de atributos. As novas colunas criadas, como categorias de sinais vitais, desfecho e erros de triagem, já estão visíveis, fornecendo um conjunto de dados enriquecido e pronto para análises exploratórias ou modelagem.

In [ ]:
display(df.head())

,Group,Sex,Age,Patients number per hour,Arrival mode,Injury,Chief_complain,Mental,Pain,NRS_pain,...,Categoria_Saturacao,Idade_Categoria,Dor_Nivel,Erro_Tipo_Triagem,Erro_Desvio_Magnitude,Permanencia_Categoria,Triagem_Velocidade,Fluxo_Categoria,KTAS_Enfermeiro_classe,KTAS_Especialista_classe
0,2,2,71,3,3,2,right ocular pain,1,1,2.0,...,Normal,Idoso,Dor Leve,Sub-triagem,Desvio Leve,Curta (até 6h),Padrão,Fluxo Baixo,Emergência,Não Urgente
1,1,1,56,12,3,2,right forearm burn,1,1,2.0,...,Normal,Adulto,Dor Leve,Sub-triagem,Desvio Grave,Curta (até 6h),Padrão,Superlotado,Não Urgente,Não Urgente
2,2,1,68,8,2,2,"arm pain, Lt",1,1,2.0,...,Normal,Idoso,Dor Leve,Sub-triagem,Desvio Grave,Média (6h-24h),Rápida,Fluxo Médio,Não Urgente,Não Urgente
3,1,2,71,8,1,1,ascites tapping,1,1,3.0,...,Normal,Idoso,Dor Leve,Sub-triagem,Desvio Grave,Curta (até 6h),Demorada,Fluxo Médio,Não Urgente,Não Urgente
4,1,2,58,4,3,1,"distension, abd",1,1,3.0,...,Normal,Adulto,Dor Leve,Sub-triagem,Desvio Grave,Curta (até 6h),Padrão,Fluxo Baixo,Não Urgente,Não Urgente
